In [2]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t

In [3]:
# Helper functions
def stripText(text):
    x =[]
    return x.append(text)
# Convert function to UDF
stripTextUDF = f.udf(lambda z: stripText(z))

In [4]:
spark = SparkSession.builder \
    .appName("Redit Summarization")\
    .master("yarn")\
    .config("spark.executor.memoryOverhead","2048")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.2.3,org.apache.hadoop:hadoop-aws:2.7.3")\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b2d9fd1c-60f7-4eeb-bc47-38d2ada48530;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.2.3 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastu

2022-04-30 19:51:28,921 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-30 19:51:32,746 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2022-04-30 19:51:53,457 WARN yarn.Client: Same path resource file:///home/ubuntu/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-3.2.3.jar added multiple times to distributed cache.
2022-04-30 19:51:53,457 WARN yarn.Client: Same path resource file:///home/ubuntu/.ivy2/jars/org.apache.hadoop_hadoop-aws-2.7.3.jar added multiple times to distributed cache.
2022-04-30 19:51:53,457 WARN yarn.Client: Same path resource file:///home/ubuntu/.ivy2/jars/com.typesafe_config-1.4.1.jar added multiple times to distributed cache.
2022-04-30 19:51:53,457 WARN yarn.Client: Sam

In [5]:
# Read data
df = spark.read.format("csv").option("header","true").load("hdfs://namenode:9000/cleaned_data.csv")

In [6]:
df_top_comments_ids = df.filter(df.link_id == df.parent_id).rdd.map(lambda x: x.link_id).collect()

In [6]:
df.columns

['created_utc',
 'ups',
 'subreddit_id',
 'link_id',
 'subreddit',
 'parent_id',
 'clean_body']

In [18]:
df_grouped = df.groupBy("created_utc","subreddit","parent_id").agg(f.sum("ups").alias("total_ups"),f.concat_ws(", ", f.collect_list(df.clean_body)).alias("comment"))

In [19]:
df_grouped.show(10, truncate = True)

+-----------+------------+----------+---------+--------------------+
|created_utc|   subreddit| parent_id|total_ups|             comment|
+-----------+------------+----------+---------+--------------------+
| 2015-05-01|    ""2B"")"|         0|      2.0|          1432718658|
| 2015-05-01|1200isplenty|t1_cqurgh2|     19.0|holy wow skinnypo...|
| 2015-05-01|1200isplenty|t1_cqv6b56|      9.0|maybe so but the ...|
| 2015-05-01|1200isplenty| t3_34gbc2|      9.0|have you tried pu...|
| 2015-05-01|   2007scape|t1_cqtywli|      3.0|its not even ridi...|
| 2015-05-01|   2007scape|t1_cqu9d1p|      1.0|obviously 50k is ...|
| 2015-05-01|   2007scape|t1_cqugx2k|      1.0|sells you a rando...|
| 2015-05-01|   2007scape|t1_cquh5fg|     -8.0|my background is ...|
| 2015-05-01|   2007scape|t1_cqujpkk|      7.0|nice reference xd...|
| 2015-05-01|   2007scape|t1_cqul8mj|      1.0| don t worry i won t|
+-----------+------------+----------+---------+--------------------+
only showing top 10 rows



In [21]:
df_grouped_max = df_grouped.groupBy("created_utc","subreddit").agg(f.max("total_ups").alias("max_ups"), f.first("parent_id").alias("parent_id"),f.first("comment").alias("comment"))

In [17]:
df_grouped_max.filter(df_grouped_max.subreddit=="AskReddit").show(10, truncate=False)

+-----------+---------+-------+----------+
|created_utc|subreddit|max_ups|parent_id |
+-----------+---------+-------+----------+
|2015-05-01 |AskReddit|60357.0|t3_34h5ef |
|2015-05-02 |AskReddit|53436.0|t3_34kv9h |
|2015-05-04 |AskReddit|41495.0|t3_34tk8t |
|2015-05-06 |AskReddit|80169.0|t3_352x0z |
|2015-05-07 |AskReddit|44391.0|t1_cr1sqy8|
|2015-05-09 |AskReddit|42990.0|t3_35czvb |
|2015-05-14 |AskReddit|49931.0|t1_cr8dwbs|
|2015-05-15 |AskReddit|30815.0|t1_cr9qd8v|
|2015-05-16 |AskReddit|45683.0|t3_3679zd |
|2015-05-18 |AskReddit|55622.0|t1_crd7jmf|
+-----------+---------+-------+----------+
only showing top 10 rows



In [23]:
df_grouped_max.write.option("header","true").csv("hdfs://namenode:9000/popular_comments.csv")